# Projet du cours "Kernel methods" du master MVA
Le projet de ce cours consiste en la classification d'images. Dans ce notebook, on implémente de manière la plus détaillée et la plus claire possible l'ensemble des classes et fonctions des algorithmes que nous utilisons.    

Ce projet se fera en 3 étapes :
- Preprocessing des données
- Feature extraction
- Prediction par SVM

Il est indiqué dans l'énoncé que l'étape de preprocessing a déjà été réalisée. Nous allons donc directement procéder à l'extraction de nos features (par HOG) et à la prédiction de la nature de nos images (par SVM).

Ce notebook utilise comme kernel le RBF kernel qui est celui qui donne les meilleurs résultats. Pour voir les implémentations réalisées avec d'autres kernels, on pourra se reporter aux notebooks joints (moins détaillés et n'effectuant pas toutes les étapes ci-dessous mais présentant notamment les implémentations liées à chaque kernel considéré).

## Chargement des modules

In [1]:
%pylab inline
import pandas
import cvxopt
import time
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt 
import scipy
from scipy.stats import mode
from scipy.linalg import eigh

Populating the interactive namespace from numpy and matplotlib


# Importation des données

On importe les données du concours

In [2]:
df_train=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xtr.csv',header=None,sep=',')
df_train=df_train.drop(3072,1)
df_test=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xte.csv',header=None,sep=',')
df_test=df_test.drop(3072,1)
y_train=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Ytr.csv',sep=';')
y_train=pandas.DataFrame(y_train['Prediction'])

In [3]:
X_train = np.genfromtxt('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xtr.csv',delimiter=',')[:,0:3072]
Y_train = np.genfromtxt('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Ytr.csv',delimiter=';')[1:5001,1]
X_test = np.genfromtxt('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xte.csv',delimiter=',')[:,0:3072]

# Répartition des labels
Nous travaillons sur un échantillon d'apprentissage dans le but de classifier des images. Vérifions que nous avons bien équirépartition des images dans l'échantillon d'apprentissage afin de pouvoir mener à bien notre modélisation

In [4]:
y_train["Prediction"].value_counts()

7    500
3    500
6    500
2    500
9    500
5    500
1    500
8    500
4    500
0    500
Name: Prediction, dtype: int64

# Implémentation des méthodes

Dans cette partie, nous implémenterons les différentes méthodes que nous testerons par la suite. Il s'agit des méthodes suivantes:
- SVM
- ACP
- HOG



## Support Vector Machine

In [5]:
class SVMTrainer(object):
    def __init__(self, c, sigma):
        self._c = c
        self.sigma=sigma
        
    def train(self, X, y):
        lagrange_multipliers = self._compute_multipliers(X, y)
        return self._construct_predictor(X, y, lagrange_multipliers)

    def _gram_matrix(self, X):
        n_samples, n_features = X.shape
        v=norm(X,2,axis=1)**2
        v2=array([v,]*n_samples)+array([v,]*n_samples).transpose()-2*dot(X.values,transpose(X.values))
        K=exp(-v2/(2*(self.sigma)**2)) 
        print("Taille de la matrice de Gram :",K.shape)
        return K

    def _construct_predictor(self, X, y, lagrange_multipliers):
        support_vector_indices=abs(lagrange_multipliers)>0
        support_multipliers = lagrange_multipliers[support_vector_indices] # alpha i non nuls
        support_vectors = X[support_vector_indices] # vecteurs supports
        support_vector_labels = y[support_vector_indices] # labels des vecteurs supports
        print("Nombre de vecteurs supports :",len(support_vectors))
        pre=SVMPredictor(sigma=self.sigma,bias=0.0,weights=support_multipliers,
             support_vectors=support_vectors,support_vector_labels=support_vector_labels).predictest(support_vectors.values)[0]
        bias = np.mean(support_vector_labels[:,0]-pre)
        print("Biais:",bias)
        return SVMPredictor(sigma=self.sigma,bias=bias,weights=support_multipliers,support_vectors=support_vectors,
            support_vector_labels=support_vector_labels)

    def _compute_multipliers(self, X, y):
        n_samples, n_features = X.shape
        K=self._gram_matrix(X)
        P=cvxopt.matrix(K)
        q=cvxopt.matrix(-1*y[:,0])
        G_std=cvxopt.matrix(-1*np.diag(y[:,0]))
        h_std=cvxopt.matrix(np.zeros(n_samples))
        G_slack=cvxopt.matrix(np.diag(y[:,0]))
        h_slack=cvxopt.matrix(np.ones(n_samples) * self._c)
        G=cvxopt.matrix(np.vstack((G_std, G_slack)))
        h=cvxopt.matrix(np.vstack((h_std, h_slack)))
        A=cvxopt.matrix(np.ones(n_samples),(1,n_samples))
        b=cvxopt.matrix(0.0)
        solution = cvxopt.solvers.qp(P, q, G, h, A=None, b=None)
        return np.ravel(solution['x'])

In [6]:
class SVMPredictor(object):
    def __init__(self,sigma,bias,weights,support_vectors,support_vector_labels):
        self.sigma=sigma
        self._bias = bias
        self._weights = weights
        self._support_vectors = support_vectors
        self._support_vector_labels = support_vector_labels

    def predictest(self,x):
        v = array([norm(x,2,axis=1)**2,]*len(self._support_vectors))
        inter = array([norm(self._support_vectors,2,axis=1)**2,]*len(x)).transpose() + v
        inter1 = inter - 2*dot(self._support_vectors,transpose(x))
        inter2 = exp(-asarray(inter1)/(2*(self.sigma)**2))
        result = dot(transpose(self._weights),inter2)+self._bias
        return sign(result),result

## Adaptation du SVM au cas multi-classe
On va tout d'abord implémenter la méthode One Vs One, i.e. considérer 10x9/2=45 problèmes de classifications binaires en traitant des paires d'instances, puis implémenter la méthode One Vs Rest, i.e. entraîner un classifieur par classe.

### One Vs One

In [7]:
def SVMOvO(data_train,data_test,is_train,C,sigma):
    SVM=SVMTrainer(C,sigma)
    result=(20*ones(len(data_test))).reshape(len(data_test),1)
    for i in [0,1,2,3,4,5,6,7,8]:
        for j in range(i+1,10):
            print("Classe :",i,j)
            df_train_ij=data_train[asmatrix(is_train,dtype=ndarray)==(i,j)].copy()
            is_train_ij=is_train[asmatrix(is_train,dtype=ndarray)==(i,j)].copy()
            is_train_ij[is_train==i]=1
            is_train_ij[is_train==j]=-1
            s1=time.clock()
            SVMPred=SVM.train(df_train_ij,asfarray(is_train_ij))
            s2=time.clock()
            print("Temps pour l'entrainement :",s2-s1)
            s3=time.clock()
            values=SVMPred.predictest(data_test.values)[0]
            s4=time.clock()
            print("Temps pour la prédiction :",s4-s3)
            values[values==1]=i
            values[values==-1]=j
            values=values.reshape(len(data_test),1)
            result=concatenate((result,values),1)
            is_test_predict=mode(result,axis=1)
    return(is_test_predict)    

### One Vs Rest

In [8]:
def SVMOvR(data_train,data_test,is_train,C,sigma):
    SVM=SVMTrainer(C,sigma)
    result=(-20*ones(len(data_test))).reshape(len(data_test),1)
    for i in [0,1,2,3,4,5,6,7,8,9]:
        print("Classe :",i)
        is_train_i=is_train.copy()
        is_train_i[is_train==i]=1
        is_train_i[is_train!=i]=-1
        s1=time.clock()
        SVMPred=SVM.train(data_train,asfarray(is_train_i))
        s2=time.clock()
        print("Temps pour l'entrainement :",s2-s1)
        s3=time.clock()
        values=SVMPred.predictest(data_test.values)[1]
        s4=time.clock()
        print("Temps pour la prédiction :",s4-s3)
        values=values.reshape(len(data_test),1)
        result=concatenate((result,values),1)
    predict=argmax(result,axis=1)-1
    return(predict)  

# HOG

Implémentation de la méthode d'extraction de features inspirée de l'article https://hal.inria.fr/inria-00548512/document de N. Dalal et B. Triggs et du site http://www.learnopencv.com/histogram-of-oriented-gradients/. On choisit de diviser l'image de 32 pixels en 4 cellules de (8,8) pixels chacun, en normalisant par blocs de 4 cellules, ce qui donne une base X de taille 3 (nombre de blocs sur l'axe x) fois 3 (nombre de blocs sur l'axe y) fois 9 (taille d'un histogramme) fois 4 (nombre d'histogrammes par bloc, un histogramme correspondant à une cellule) fois 3 (pour les différentes couleurs de l'image), i.e. on obtient une base X avec 972 variables au lieu de 3072.

In [10]:
class HOG :

    def __init__(self, nbins=9):
        self.nbins = nbins

    def _calc_gradient_discret(self, img):
        n_x, n_y = img.shape
        histogram = numpy.zeros((4, 4, self.nbins))

        for i in range(0, n_x) :
            for j in range(0, n_y) :
                dx = 0
                dy = 0
                if i < n_x - 1 :
                    dx += img[i + 1, j]
                if i > 0 :
                    dx -= img[i - 1, j]
                if j < n_y - 1 :
                    dy += img[i, j + 1]
                if j > 0 :
                    dy -= img[i, j - 1]

                if dy == 0 and dx == 0 :
                    continue

                magnitude = numpy.sqrt(dx**2 + dy**2)
                if dx == 0 :
                    angle = numpy.pi / 2
                else:
                    angle = numpy.arctan(dy / dx)
                    angle = (angle + numpy.pi / 2) / (numpy.pi / self.nbins)
               
                bin_pos = int(numpy.floor(angle))

                if bin_pos == self.nbins :
                    bin_pos = 0
                    angle = 0

                closest_bin = bin_pos

                if bin_pos == 0 :
                    if angle < 0.5 :
                        second_closest_bin = self.nbins - 1
                    else:
                        second_closest_bin = 1
                elif bin_pos == self.nbins - 1 :
                    if angle < self.nbins - 0.5 :
                        second_closest_bin = self.nbins - 2
                    else:
                        second_closest_bin = 0
                else:
                    if angle < bin_pos + 0.5 :
                        second_closest_bin = bin_pos - 1
                    else:
                        second_closest_bin = bin_pos + 1

                if angle < bin_pos + 0.5 :
                    second_closest_bin_distance = angle - (bin_pos - 0.5)
                else:
                    second_closest_bin_distance = (bin_pos + 1.5) - angle

                r = second_closest_bin_distance
                histogram[i / 8, j / 8, closest_bin] += r * magnitude
                histogram[i / 8, j / 8, second_closest_bin] += (1 - r) * magnitude

        vec = numpy.zeros((3, 3, self.nbins * 4))

        for i in range(3):
            for j in range(3):
                aux = histogram[i:i + 2, j:j + 2, :].flatten().copy()
                aux = aux / numpy.linalg.norm(aux)
                vec[i, j, :] = aux

        return vec.flatten()

    
    def _calc_gradient_one_image(self, img):
        nchannels = img.shape[2]
        vec = []

        for i in range(nchannels):
            vec.append(self._calc_gradient_discret(img[:,:,i]))

        return numpy.array(vec).flatten()

    
    def hog(self, X):
        n = X.shape[0]
        X_new = []

        for i in range(n):
            X_new.append(self._calc_gradient_one_image(X[i,:,:,:]))

        return numpy.array(X_new)

# Tests
Afin de tester nos modèles sur nos données d'entraînement, on va découper notre base d'entraînement en train/test à hauteur de 60/40%.

Pour plus de clarté, on ne présente ici que le modèle final. Se reporter aux autres notebooks pour plus de détails.

### Après HOG :  SVM One Vs Rest (C=1, sigma=2) 

In [12]:
# Création de la nouvelle base X obtenue après extraction de features

Xtrain = numpy.reshape(X_train, (X_train.shape[0], 3, 32, 32))
Xtrain = numpy.swapaxes(Xtrain, 1, 2)
Xtrain = numpy.swapaxes(Xtrain, 2, 3)

Xtest = numpy.reshape(X_test, (X_test.shape[0], 3, 32, 32))
Xtest = numpy.swapaxes(Xtest, 1, 2)
Xtest = numpy.swapaxes(Xtest, 2, 3)

histogram=HOG(nbins=9)
h=histogram.hog(Xtrain)
g=histogram.hog(Xtest)

X_tr = pandas.DataFrame(h)
X_te = pandas.DataFrame(g)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:63: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:64: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [16]:
X_train_2 = X_tr[0:3000]
X_test_2 = X_tr[3000:5000]
Y_train_2 = y_train[0:3000]
Y_test_2 = y_train[3000:5000]
print(X_train_2.shape)
print(X_test_2.shape)
Y_train_2["Prediction"].value_counts()

(3000, 972)
(2000, 972)


3    311
1    308
0    306
6    305
8    304
4    298
5    294
7    292
9    291
2    291
Name: Prediction, dtype: int64

In [17]:
y_predict_OvR=SVMOvR(X_train_2,X_test_2,Y_train_2,1,2)

Classe : 0
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -4.1590e+02 -5.3848e+03  2e+04  2e+00  2e-15
 1: -3.4990e+02 -2.9285e+03  4e+03  2e-01  2e-15
 2: -3.3254e+02 -8.3656e+02  6e+02  2e-02  3e-15
 3: -3.6704e+02 -5.1380e+02  2e+02  5e-03  2e-15
 4: -3.8693e+02 -4.2974e+02  4e+01  1e-03  2e-15
 5: -3.9489e+02 -4.0666e+02  1e+01  5e-05  2e-15
 6: -3.9776e+02 -3.9985e+02  2e+00  4e-06  2e-15
 7: -3.9840e+02 -3.9862e+02  2e-01  3e-07  2e-15
 8: -3.9848e+02 -3.9849e+02  7e-03  5e-09  2e-15
 9: -3.9849e+02 -3.9849e+02  2e-04  9e-11  2e-15
Optimal solution found.
Nombre de vecteurs supports : 3000
Biais: 0.046
Temps pour l'entrainement : 43.387455000000045
Temps pour la prédiction : 1.0714130000000068
Classe : 1
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -4.0098e+02 -5.6251e+03  3e+04  2e+00  3e-15
 1: -3.3132e+02 -3.1783e+03  4e+03  2e-01  3e-15
 2: -3.1044e+02 -8.8245e+02  7e+02  3e-02

In [18]:
compt=Y_test_2.values[:,0]-y_predict_OvR
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_2))

Pourcentage de bonnes prédictions : 0.5625


In [18]:
compt=Y_test_3.values[:,0]-y_predict_OvR
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_3))

Pourcentage de bonnes prédictions : 0.3785


# On soumet nos résultats finaux sur le site Kaggle

### Le modèle optimal est un SVM One Vs Rest (paramètres C=1 et sigma=2) après HOG mais sans ACP

In [23]:
final_pred=SVMOvR(X_tr,X_te,y_train,1,2)

Classe : 0
Taille de la matrice de Gram : (5000, 5000)
     pcost       dcost       gap    pres   dres
 0:  1.8483e+02 -8.3378e+04  2e+05  5e-01  1e-14
 1:  3.2853e+02 -2.1738e+04  3e+04  5e-02  1e-14
 2: -2.5897e+02 -5.7085e+03  6e+03  9e-03  1e-14
 3: -6.4948e+02 -2.6656e+03  2e+03  2e-03  9e-15
 4: -7.9547e+02 -1.6355e+03  8e+02  3e-04  9e-15
 5: -8.6431e+02 -1.1129e+03  2e+02  2e-16  9e-15
 6: -8.9280e+02 -9.6193e+02  7e+01  2e-16  9e-15
 7: -9.0331e+02 -9.1961e+02  2e+01  2e-16  9e-15
 8: -9.0677e+02 -9.0849e+02  2e+00  2e-16  9e-15
 9: -9.0724e+02 -9.0731e+02  6e-02  2e-16  9e-15
10: -9.0726e+02 -9.0726e+02  2e-03  2e-16  9e-15
11: -9.0726e+02 -9.0726e+02  5e-05  2e-16  9e-15
Optimal solution found.
Nombre de vecteurs supports : 5000
Biais: 0.0
Temps pour l'entrainement : 229.1030400000002
Temps pour la prédiction : 1.4648799999999937
Classe : 1
Taille de la matrice de Gram : (5000, 5000)
     pcost       dcost       gap    pres   dres
 0:  6.2078e+02 -8.9389e+04  2e+05  5e-01  1

In [24]:
results = [int(i) for i in final_pred]
X_result = pandas.DataFrame(results,columns=['Prediction'])
X_result.index += 1
X_result.to_csv('Yte.csv',index=True,index_label='Id')